## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Port Alfred,ZA,-33.5906,26.8910,59.85,81,92,10.71,Clouds,2021-05-27 23:23:42
1,1,Hobyo,SO,5.3505,48.5268,78.40,82,57,18.84,Clouds,2021-05-27 23:23:43
2,2,Cabo San Lucas,MX,22.8909,-109.9124,96.51,53,61,10.83,Clouds,2021-05-27 23:23:27
3,3,Provideniya,RU,64.3833,-173.3000,46.51,57,0,15.66,Clear,2021-05-27 23:23:05
4,4,Vaini,TO,-21.2000,-175.2000,77.16,69,20,11.50,Clouds,2021-05-27 23:23:05


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_citys_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_citys_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
1,1,Hobyo,SO,5.3505,48.5268,78.40,82,57,18.84,Clouds,2021-05-27 23:23:43
4,4,Vaini,TO,-21.2000,-175.2000,77.16,69,20,11.50,Clouds,2021-05-27 23:23:05
11,11,Kapaa,US,22.0752,-159.3190,84.18,70,98,3.00,Clouds,2021-05-27 23:23:07
13,13,Caiaponia,BR,-16.9567,-51.8103,76.89,47,45,1.90,Clouds,2021-05-27 23:23:13
15,15,Riohacha,CO,11.5444,-72.9072,82.26,73,100,10.60,Clouds,2021-05-27 23:23:08
24,24,Kimbe,PG,-5.5502,150.1429,82.17,74,93,6.69,Clouds,2021-05-27 23:23:11
29,29,Birao,CF,10.2849,22.7882,77.25,65,49,4.43,Clouds,2021-05-27 23:23:13
35,35,Kota Kinabalu,MY,5.9749,116.0724,82.38,83,20,4.61,Clouds,2021-05-27 23:23:15
46,46,Mitsamiouli,KM,-11.3847,43.2844,76.10,58,3,4.00,Clear,2021-05-27 23:23:18
49,49,Luwuk,ID,-0.9516,122.7875,82.44,73,17,5.48,Clouds,2021-05-27 23:23:19


In [8]:
# 4a. Determine if there are any empty rows.
preferred_citys_df.isnull().sum()

City ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Date           0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_citys_df.dropna(how= 'any')

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hobyo,SO,78.40,Clouds,5.3505,48.5268,
4,Vaini,TO,77.16,Clouds,-21.2000,-175.2000,
11,Kapaa,US,84.18,Clouds,22.0752,-159.3190,
13,Caiaponia,BR,76.89,Clouds,-16.9567,-51.8103,
15,Riohacha,CO,82.26,Clouds,11.5444,-72.9072,
24,Kimbe,PG,82.17,Clouds,-5.5502,150.1429,
29,Birao,CF,77.25,Clouds,10.2849,22.7882,
35,Kota Kinabalu,MY,82.38,Clouds,5.9749,116.0724,
46,Mitsamiouli,KM,76.10,Clear,-11.3847,43.2844,
49,Luwuk,ID,82.44,Clouds,-0.9516,122.7875,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... Skipping")
        

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hobyo,SO,78.40,Clouds,5.3505,48.5268,Halane Hotel and Restaurent
4,Vaini,TO,77.16,Clouds,-21.2000,-175.2000,Keleti Beach Resort
11,Kapaa,US,84.18,Clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
13,Caiaponia,BR,76.89,Clouds,-16.9567,-51.8103,HOTEL PALACE AVENIDA & CAIAPÓ TURISMO
15,Riohacha,CO,82.26,Clouds,11.5444,-72.9072,Hotel Taroa
24,Kimbe,PG,82.17,Clouds,-5.5502,150.1429,Kimbe Bay Hotel
29,Birao,CF,77.25,Clouds,10.2849,22.7882,
35,Kota Kinabalu,MY,82.38,Clouds,5.9749,116.0724,Gaya Centre Hotel
46,Mitsamiouli,KM,76.10,Clear,-11.3847,43.2844,Foyer ADM
49,Luwuk,ID,82.44,Clouds,-0.9516,122.7875,Hotel Santika Luwuk


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = 'any')

In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))